In [9]:
from sklearn.tree import _tree

In [10]:
def tree_to_json(tree):
    # Accessing the internal tree structure
    tree_ = tree.tree_
    
    # Defining a placeholder list for feature names (assuming 30 features)
    feature_names = range(30)
    
    # Creating a list of feature names where each node has its feature name or 'undefined!' if it's a leaf node
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    # Recursive function to traverse the tree and convert it into JSON format
    def recurse(node):
        tree_json = dict()  # Initialize an empty dictionary to hold the JSON structure
        
        # Check if the current node is a decision (split) node
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            tree_json['type'] = 'split'  # Mark this node as a split
            threshold = tree_.threshold[node]  # Get the threshold for this node
            tree_json['threshold'] = "{} <= {}".format(feature_name[node], threshold)  # Store threshold info
            
            # Recursively process the left child
            tree_json['left'] = recurse(tree_.children_left[node])
            
            # Recursively process the right child
            tree_json['right'] = recurse(tree_.children_right[node])
        
        # If the node is a leaf node (no decision to make), store its value
        else:
            tree_json['type'] = 'leaf'  # Mark this node as a leaf
            tree_json['value'] = tree_.value[node].tolist()  # Store the leaf value
        
        return tree_json  # Return the constructed JSON object

    # Start the recursion from the root node (node 0)
    return recurse(0)


In [11]:
def forest_to_json(forest):
    # Initialize an empty dictionary to store the JSON structure of the forest
    forest_json = dict()

    # Store the number of features used in the forest
    forest_json['n_features'] = forest.n_features_
    
    # Store the number of classes in the classification problem (if applicable)
    forest_json['n_classes'] = forest.n_classes_
    
    # Store the list of class labels (as a list for JSON serialization)
    forest_json['classes'] = forest.classes_.tolist()
    
    # Store the number of output values (usually 1 for single-output problems)
    forest_json['n_outputs'] = forest.n_outputs_
    
    # Store the number of decision trees (estimators) in the forest
    forest_json['n_estimators'] = forest.n_estimators
    
    # Convert each individual decision tree (estimator) in the forest into JSON format
    # This uses the previously defined `tree_to_json` function
    forest_json['estimators'] = [tree_to_json(estimator) for estimator in forest.estimators_]
    
    # Return the complete JSON structure representing the entire forest
    return forest_json
